<a href="https://colab.research.google.com/github/anruki/UFV_Visualizacion/blob/main/EjerciciosClase/Ventas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicciones de ventas
### Clase 18/10/2024
### Asignatura: visualización de datos




### Importación de librerías:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

### Preguntas a responder para entender los datos:
(Antes de responder)


1.   NºActuals y Forecasts
2.   Horizonte de predicción
Cuando haces una predicción, ej: estás en enero:

  *  predices x ventas para febrero, entonces horizonte de predicción = 1
  *  predices x ventas en febrero, en marzo y, en abril z, entonces horizonte de predicción = 3.

3.  Nº Países y de productos
4.  Histórico de actuals/forecasts
5.  Forecasts distintos


In [8]:
# URL del archivo CSV en GitHub
url = "https://github.com/anruki/UFV_Visualizacion/blob/703c84e6d7d6424dd7e27fd3ebc2a47b8c29f414/EjerciciosClase/datos_ejercicio_ventas.csv"
nombre = 'datos_ejercicios_ventas.csv'


# Realizar la solicitud GET para obtener el contenido del archivo
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
     # Guardar el contenido del archivo en un archivo local
    with open(nombre, 'wb') as f:
        f.write(response.content)
    print("Archivo descargado exitosamente.")
else:
    print("Error al descargar el archivo.")

Error al descargar el archivo.
